In [3]:
%pip install wandb

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!wandb login 10713c140f304d47b99c544f0f74e73b7d667cbf

wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Administrator\_netrc


In [3]:
# This script needs these libraries to be installed:
#   numpy, transformers, datasets
import wandb

import os
import numpy as np
from datasets import load_dataset
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForSequenceClassification


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": np.mean(predictions == labels)}


# download prepare the data
dataset = load_dataset("yelp_review_full")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

small_train_dataset = dataset["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = dataset["test"].shuffle(seed=42).select(range(300))

small_train_dataset = small_train_dataset.map(tokenize_function, batched=True)
small_eval_dataset = small_train_dataset.map(tokenize_function, batched=True)

f:\test\chatbot\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Map: 100%|██████████| 1000/1000 [00:00<00:00, 5426.76 examples/s]


In [4]:
# download the model
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=5)

# set the wandb project where this run will be logged
os.environ["WANDB_PROJECT"]="my-model-project"

# save your trained model checkpoint to wandb
os.environ["WANDB_LOG_MODEL"]="true"

# turn off watch to log faster
os.environ["WANDB_WATCH"]="false"

# pass "wandb" to the 'report_to' parameter to turn on wandb logging
training_args = TrainingArguments(
    output_dir='models',
    report_to="wandb",
    logging_steps=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="steps",
    eval_steps=20,
    max_steps = 100,
    save_steps = 100
)

# define the trainer and start training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
f:\test\chatbot\env\Lib\site-packages\transformers\training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code

  5%|▌         | 5/100 [02:35<48:31, 30.65s/it]

{'loss': 1.606, 'grad_norm': 1.8802423477172852, 'learning_rate': 4.75e-05, 'epoch': 0.16}


 10%|█         | 10/100 [05:11<46:01, 30.69s/it]

{'loss': 1.5961, 'grad_norm': 1.2873133420944214, 'learning_rate': 4.5e-05, 'epoch': 0.31}


...

{'eval_loss': 0.7370273470878601, 'eval_accuracy': 0.743, 'eval_runtime': 322.664, 'eval_samples_per_second': 3.099, 'eval_steps_per_second': 0.099, 'epoch': 3.12}


100%|██████████| 100/100 [1:18:04<00:00, 29.64s/it]max_steps is given, it will override any value given in num_train_epochs


{'train_runtime': 4690.3566, 'train_samples_per_second': 0.682, 'train_steps_per_second': 0.021, 'train_loss': 1.0882170462608338, 'epoch': 3.12}


100%|██████████| 100/100 [1:18:05<00:00, 46.86s/it]


eval/accuracy,▁▅▆██
eval/loss,█▄▃▁▁
eval/runtime,▆██▁▅
eval/samples_per_second,▃▁▁█▄
eval/steps_per_second,▄▁▁█▄
train/epoch,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▅▅▆▆▇▇▇▇████
train/global_step,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▅▅▆▆▇▇▇▇████
train/grad_norm,▂▁▂▄▄█▃▃▅▅▅▅███▅▄▃█▄
train/learning_rate,██▇▇▇▆▆▅▅▅▄▄▄▃▃▂▂▂▁▁
train/loss,██▇▇▆▅▄▄▄▄▃▃▂▂▂▂▁▂▂▁
eval/accuracy,0.743


In [12]:
import wandb
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
import torch

# Initialize Wandb run
run = wandb.init()

# Sử dụng artifact để tải mô hình
artifact = run.use_artifact('manhtien310701-fanyuan/my-model-project/model-vd07t9u5:v1', type='model')
artifact_dir = artifact.download()

# Load tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Load mô hình từ thư mục đã tải xuống
model = DistilBertForSequenceClassification.from_pretrained(artifact_dir)

# Chuẩn bị dữ liệu đầu vào
input_text = "Good fish sandwich."
inputs = tokenizer(input_text, return_tensors='pt')

# Dự đoán với mô hình
with torch.no_grad():
    outputs = model(**inputs)
    predictions = outputs.logits.argmax(dim=-1)

# In kết quả dự đoán
print(f"Predicted label: {predictions.item()}")

wandb: Downloading large artifact model-vd07t9u5:v1, 255.44MB. 4 files... 
wandb:   4 of 4 files downloaded.  
Done. 0:0:0.7


Predicted label: 3
